This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "cpi"

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [5]:
# The get_series_info method may be useful
fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Sticky Core": "CORESTICKM159SFRBATL",
    "Core": "CPILFESL",
    "Headline": "CPIAUCSL"
}

In [6]:
today = dt.date.today()

core_sticky_srs = fred.get_series(
    series_id=series_codes["Sticky Core"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Sticky Core")

core_srs = fred.get_series(
    series_id=series_codes["Core"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    units='pc1'
).rename("Core")

headline_srs = fred.get_series(
    series_id=series_codes["Headline"],
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    units='pc1'
).rename("Headline")

joined_df = pd.concat(
    [core_sticky_srs, core_srs, headline_srs], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df.tail()

,Sticky Core,Core,Headline
Date,,,
2025-04-01,3.176690,2.78152,2.33375
2025-05-01,3.159131,2.76715,2.37593
2025-06-01,3.288774,2.90787,2.67268
2025-07-01,3.419811,3.04860,2.73180
2025-08-01,3.400000,3.11219,2.93922


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [8]:
# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'CPI Inflation <br><sup>Monthly, Seasonally Adjusted </sup>'),
    xaxis_title="Date",
    yaxis_title="Change from Year Ago (%)",
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".2f%",
    ticksuffix="%",
    range=[-.5, 10]
) 

# Again, graph specific, we have a multiyear series and want tick marks to show months and years
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',  # e.g., Jan\n2020
)

ut.add_end_labels(fig = fig)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")